In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv('data/movies_train.csv')
test = pd.read_csv('data/movies_test.csv')

In [2]:
# 결측치가 많은 데이터 제거
train = train.drop(['dir_prev_bfnum'],axis = 1)
test =  test.drop(['dir_prev_bfnum'],axis = 1)

# 감독명 : 너무 다양해서 제거
train = train.drop(['director'],axis = 1)
test = test.drop(['director'],axis = 1)

# 제목 : 의미가 없기 때문에 제거
train = train.drop(['title'],axis= 1)
test = test.drop(['title'],axis= 1)

In [3]:
train.distributor.value_counts()

CJ 엔터테인먼트        54
롯데엔터테인먼트         52
(주)NEW           30
(주)마운틴픽쳐스        29
(주)쇼박스           26
                 ..
OAL(올)            1
(주)에이원 엔터테인먼트     1
(주)콘텐츠 윙          1
위더스필름             1
퍼스트런              1
Name: distributor, Length: 169, dtype: int64

In [4]:
# 상위 5개의 배급사를 제외하고 '기타'로처리
distributor_list = train.distributor.value_counts()[:5]
def func(distributor):
    if distributor in distributor_list:
        return distributor
    else:
        return '기타'

train['distributor'] = train['distributor'].apply(lambda x : func(x))
test['distributor'] = test['distributor'].apply(lambda x : func(x))

# 개봉일을 바탕으로 년,월 변수 생성
train['년'] = train['release_time'].apply(lambda x: int(x[:4]))
train['월'] = train['release_time'].apply(lambda x: int(x[5:7]))
train =  train.drop(['release_time'],axis = 1)

test['년'] = test['release_time'].apply(lambda x: int(x[:4]))
test['월'] = test['release_time'].apply(lambda x: int(x[5:7]))
test =  test.drop(['release_time'],axis = 1)


In [5]:
train.shape, test.shape

((600, 10), (243, 9))

In [6]:
train.columns

Index(['distributor', 'genre', 'time', 'screening_rat', 'dir_prev_num',
       'num_staff', 'num_actor', 'box_off_num', '년', '월'],
      dtype='object')

In [7]:
# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

### 모델정의 및 학습

In [8]:
train_x = train.drop(['box_off_num'],axis= 1)
train_y = train['box_off_num']

model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)

RandomForestRegressor()

In [16]:
model.score(train_x, train_y)

0.8970434506365743

In [10]:
train_y.shape

(600,)

In [11]:
test.shape

(243, 28)

### 학습 된 모델로 예측 데이터 생성


In [12]:
pred = model.predict(test)

### 제출파일 생성

In [13]:
submission = pd.read_csv('data/submission.csv')
submission

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0
...,...,...
238,해에게서 소년에게,0
239,울보 권투부,0
240,어떤살인,0
241,말하지 못한 비밀,0


In [14]:
submission['box_off_num'] = pred
submission

,title,box_off_num
0,용서는 없다,2858039.68
1,아빠가 여자를 좋아해,1507000.90
2,하모니,1835115.97
3,의형제,2090829.07
4,평행 이론,1038499.36
...,...,...
238,해에게서 소년에게,145494.08
239,울보 권투부,3112.59
240,어떤살인,320969.74
241,말하지 못한 비밀,8577.21


In [15]:
# submission.to_csv('dacon_baseline_code.csv',index = False)